### 1. 데이터 로드

In [1]:
from datasets import load_dataset
import pandas as pd

news_dataset = load_dataset('nlpHakdang/aihub-news30k',  data_files={"train":"news_train_1_1.csv", "valid":"news_valid_1_1.csv"}, use_auth_token='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')

Using custom data configuration nlpHakdang___aihub-news30k-4ae9b68bff3cf5c2
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___aihub-news30k-4ae9b68bff3cf5c2/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
dart_dataset = load_dataset('nlpHakdang/beneficiary',  data_files="dart_ver1_2.csv", use_auth_token='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')

Using custom data configuration nlpHakdang___beneficiary-28d7ba8a9c412e49


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/67.0M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___beneficiary-28d7ba8a9c412e49/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

## 2. 전처리

In [3]:
dart_dataset['train']

Dataset({
    features: ['Unnamed: 0', 'stock_code', 'report_idx', 'corp_code', 'corp_name', 'report_date', '회사의 개요', '회사의 연혁', '사업의 개요', '주요 제품 및 서비스', '주요계약 및 연구개발활동'],
    num_rows: 2318
})

In [4]:
dart_dict = {}
error =0
for dart in dart_dataset['train']:
    try:
        dart_dict[dart['사업의 개요'] + " " + dart['주요 제품 및 서비스']] = dart['기업 이름']
    except:
        error+=1
error        

2318

In [5]:
import pandas as pd

length =500
result = []
cnt = 0
for e in news_dataset['train']:
    if e['category'] in ['경제','기업']:
        for i in range(0, len(e), length):
            result.append(e['article'][i:i+length])
            break
        cnt += 1
    if cnt==1000:
        break

#for e in news_dataset['train']['article'][:1000]:

df = pd.DataFrame(result)
df.to_csv('./article_train.csv', index=False, header=False)
df

,0
0,중국 투자의향기업 직접 방문투자협약 체결 협의이성훈 여수광양항만공사사장 방희석는 1...
1,이성훈 포스코 광양제철소소장 김학동가 자동차 강판 생산공장인 제3냉연공장을 최신 설...
2,이성훈 여수광양항만공사사장 방희석 이하 공사는 2018년도 승진 및 전보 인사를 단...
3,지역 특화작목틈새소득작목 발굴 미래농업 준비이성훈 광양시는 경쟁력을 갖춘 풍요로운 ...
4,2만2500여명 참여생명나눔 확산이성훈 포스코 광양제철소소장 김학동가 사랑나눔 헌혈...
...,...
995,GIST 이광희 교수 연구팀 초박막금속 투명전극 이용실생활서 접할 것으로 기대국제학...
996,5년간 다문화 60가족 총 200여명 기회 제공광주은행은 지난 10일 본점 3층에서...
997,2개월 연속 고용률 통계 작성 후 최고치 기록취업자 전년 동월比 광주 6천전남 1만...
998,김장철 수요 증가와 저장비용 상승 등으로 깐마늘과 양파 등 양념류 가격이 오름세를 ...


In [6]:
import pandas as pd

length =500
result = []
cnt = 0
for e in news_dataset['train']:
    if e['category'] in ['경제','기업']:
        result.append(e['article'])
        cnt += 1
    if cnt==1000:
        break

#for e in news_dataset['train']['article'][:1000]:

df2 = pd.DataFrame(result)
df2.to_csv('./article_train.csv', index=False, header=False)
df2

,0
0,중국 투자의향기업 직접 방문투자협약 체결 협의이성훈 여수광양항만공사사장 방희석는 1...
1,이성훈 포스코 광양제철소소장 김학동가 자동차 강판 생산공장인 제3냉연공장을 최신 설...
2,이성훈 여수광양항만공사사장 방희석 이하 공사는 2018년도 승진 및 전보 인사를 단...
3,지역 특화작목틈새소득작목 발굴 미래농업 준비이성훈 광양시는 경쟁력을 갖춘 풍요로운 ...
4,2만2500여명 참여생명나눔 확산이성훈 포스코 광양제철소소장 김학동가 사랑나눔 헌혈...
...,...
995,GIST 이광희 교수 연구팀 초박막금속 투명전극 이용실생활서 접할 것으로 기대국제학...
996,5년간 다문화 60가족 총 200여명 기회 제공광주은행은 지난 10일 본점 3층에서...
997,2개월 연속 고용률 통계 작성 후 최고치 기록취업자 전년 동월比 광주 6천전남 1만...
998,김장철 수요 증가와 저장비용 상승 등으로 깐마늘과 양파 등 양념류 가격이 오름세를 ...


## 2. NER

In [7]:
import tokenization_kobert
import utils

In [8]:
from utils import init_logger, load_tokenizer, get_labels
from predict import *

### 2.1 Evaluation

In [9]:
def predict(pred_config):
    # load model and args
    args = get_args(pred_config)
    device = get_device(pred_config)
    model = load_model(pred_config, args, device)
    label_lst = get_labels(args)
    logger.info(args)

    # Convert input file to TensorDataset
    pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index
    tokenizer = load_tokenizer(args)
    lines = read_input_file(pred_config)
    dataset = convert_input_file_to_tensor_dataset(lines, pred_config, args, tokenizer, pad_token_label_id)

    # Predict
    sampler = SequentialSampler(dataset)
    data_loader = DataLoader(dataset, sampler=sampler, batch_size=pred_config.batch_size)

    all_slot_label_mask = None
    preds = None

    for batch in tqdm(data_loader, desc="Predicting"):
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {"input_ids": batch[0],
                      "attention_mask": batch[1],
                      "labels": None}
            if args.model_type != "distilkobert":
                inputs["token_type_ids"] = batch[2]
            outputs = model(**inputs)
            logits = outputs[0]

            if preds is None:
                preds = logits.detach().cpu().numpy()
                all_slot_label_mask = batch[3].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                all_slot_label_mask = np.append(all_slot_label_mask, batch[3].detach().cpu().numpy(), axis=0)

    preds = np.argmax(preds, axis=2)
    slot_label_map = {i: label for i, label in enumerate(label_lst)}
    print(slot_label_map)
    preds_list = [[] for _ in range(preds.shape[0])]

    for i in range(preds.shape[0]):
        for j in range(preds.shape[1]):
            if all_slot_label_mask[i, j] != pad_token_label_id:
                preds_list[i].append(slot_label_map[preds[i][j]])
    
    # Write to output file
    with open(pred_config.output_file, "w", encoding="utf-8") as f:
        for words, preds in zip(lines, preds_list):
            #line = ""
            line = []
            for word, pred in zip(words, preds):
                if pred == 'O' or pred =='UNK':
                    #line = line + word + " "
                    pass
                else:
                    #line = line + "[{}:{}] ".format(word, pred)
                    if not "기자" in word:
                        #line.append([word, pred])
                        line.append(word)

            f.write("{}\n".format(line))

    logger.info("Prediction Done!")

In [10]:
parser = argparse.ArgumentParser()

parser.add_argument("--input_file", default="article_train.csv", type=str, help="Input file for prediction")
parser.add_argument("--output_file", default="sample_pred_out.txt", type=str, help="Output file for prediction")
parser.add_argument("--model_dir", default="./model", type=str, help="Path to save, load model")

parser.add_argument("--batch_size", default=32, type=int, help="Batch size for prediction")
parser.add_argument("--no_cuda", action="store_false", default=False, help="Avoid using CUDA when available")

pred_config = parser.parse_args(args=[])

In [11]:
predict(pred_config)

./model
<class 'transformers.models.auto.tokenization_auto.AutoTokenizer'> klue/roberta-large


Predicting: 100%|██████████| 32/32 [00:11<00:00,  2.71it/s]


{0: 'UNK', 1: 'O', 2: 'FLD-B', 3: 'FLD-I', 4: 'AFW-B', 5: 'AFW-I', 6: 'ORG-B', 7: 'ORG-I', 8: 'CVL-B', 9: 'CVL-I', 10: 'EVT-B', 11: 'EVT-I', 12: 'TRM-B', 13: 'TRM-I'}


### 2.2. 예측결과 불러오기

In [12]:
ner_list = []
with open("./sample_pred_out.txt", "r") as f:
    for line in f.readlines():
        ner_list.append(line)

In [13]:
ner_list[0:100]

["['여수광양항만공사사장', 'CEO가', '중남부지역에서', '펼친다광양만권경제자유구역청과', '사장이', '사장은', '푸드존', '운남허메이격치주를', '콜드체인', '냉동냉장창고', '바이란식품주을']\n",
 "['포스코', '광양제철소소장', '자동차', '제3냉연공장을', '자동차', '3냉연공장', '자동차', '자동차', '자동차']\n",
 "['여수광양항만공사사장', '공사는', '글로벌마케팅팀', '물류단지팀의', '정부', '1급실팀장', '△항만운영팀장', '2급부장', '△항만개발팀장', '△재무회계팀', '△물류단지팀', '3급차장', '△가치경영팀', '△인재육성팀', '△기획조정실', '△항만시설팀', '◇', '4급과장', '△재무회계팀']\n",
 "['생강', '취나물', '수박', '알스트로메리아', '아열대소득과수', '애플망고']\n",
 "['포스코', '광양제철소소장', '헌혈행사는', '광양제철소와', '대한적십자사', '광주전남혈액원이', '광양제철소에서', '임직원들과', '시민들의', '메일', '직원들의', '직원들에게', '임직원들이']\n",
 "['인공지능빅데이터스마트팩토리', '포스코', '광양제철소소장', '인공지능', '빅데이터', '스마트팩토리구현과', '임직원', '포스코의', '사물인터넷', '빅데이터', '인공지능', '직원이', '임직원들은', '스마트팩토리']\n",
 "['동광양농협조합장', '하나로마트가', '대회의실에서', '정기총회를', '하나로마트를', '조합장은지난', '임직원이']\n",
 "['광양상공회의소', '회장을', '광양시청을', '시장과', '시장은', '회장의', '상의회장은기업이']\n",
 "['부영그룹의', '회장이', '광양읍민들의', '회장은', '특정범죄가중처벌법상', '공정거래법', '임대주택법', '회장의', '회장의']\n",
 "['이에스바이오', '농업회사법인주식회사이하', '㈜이에스바이오대표', '시민접견실에서', '협약이다협약식에는'

## 3. BM25 문서검색

In [14]:
contents_list = ['회사의 개요',  '사업의 개요', '주요 제품 및 서비스','주요계약 및 연구개발활동']

dart_dict= {
    "corp_name" :[],
    "corp_text":[]
}

for corp in dart_dataset['train']:
    dart_text = ""
    dart_dict['corp_name'].append(corp['corp_name'])
    for content in contents_list:
        dart_text += corp[content]
    dart_dict['corp_text'].append(dart_text)

In [15]:

news_list = []
for news in tqdm(news_dataset['train']):
    if news['category'] == '경제' or news['category'] =='기업':
        news_list.append(news['article'])

split_news=[]
for article in news_list:
    split_news.append(article.split("."))


100%|██████████| 271093/271093 [00:40<00:00, 6663.35it/s]


In [16]:
import re
def preprocessing(s): 
        hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')
        result = hangul.sub('', s)
        return result

In [17]:
pre_dart = {}
for idx in range(len(dart_dict['corp_name'])):
    pre_dart[dart_dict['corp_name'][idx]] = preprocessing(dart_dict['corp_text'][idx])
    
pre_name = {}
for idx in range(len(dart_dict['corp_name'])):
    pre_name[preprocessing(dart_dict['corp_text'][idx])] = dart_dict['corp_name'][idx]


In [18]:
from konlpy.tag import Mecab

tokenizer = Mecab()
tot = []
for txt in tqdm(list(pre_dart.values())):
    tot.append(tokenizer.nouns(txt))

100%|██████████| 2318/2318 [00:40<00:00, 57.75it/s]


In [19]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tot)

In [20]:
ner_list[3]

"['생강', '취나물', '수박', '알스트로메리아', '아열대소득과수', '애플망고']\n"

### 3.1. NER with BM25

In [21]:
def print_corp(newsindex, dart_dict, bm25, pre_dart, pre_name):
    target_idx=newsindex
    target = df.iloc[target_idx]
    inside_corp = []

    stopwords=['모두','지난','있다','회의','강조','성과','오는','모집','과정','계약',"체결","건립",'현대식','사업',
            '주관사로','통해','일까지','이벤트','지원','지역','기자','실험','한다','진행','설명회','예정이다',
            '이번','방문','올해','제공','찾아가','재림','당시','한국','프로젝트','있는','추진','다양한','적극',
            '위해','나갈','협력','협약','상당의','물품','명으로','지난해','증가했다','의한','영향','보면','통계',
            '순위','명당','관련','업체','부실','관심','명칭','합작법인','일자리','시흥배곧','서울대','했다',
            '한다고','위원장은','기반','혁신','인재','마련','정책','권고안','함께','시장','성장','대비','전월',
            '전년','동월','물가','하락','소비자','상승','운영','증가','억원','억만원','할인','사용','것으']

    tokenized_query = tokenizer.nouns(ner_list[newsindex])
    #print('tokenized_query:',tokenized_query)

    for key in ' '.join(target).split(" "):
        if key in dart_dict['corp_name']:
            inside_corp.append(key)

    for key in tokenized_query:
        if key in dart_dict['corp_name']:
            inside_corp.append(key)
    
    inside_corp = list(set(inside_corp))
    
    doc_scores = bm25.get_scores(tokenized_query)
    print(doc_scores)
    print(max(doc_scores))
    if max(doc_scores) <3:
        #마땅한 회사가 없음
        return False
    else:
        list_dart_n = bm25.get_top_n(tokenized_query, list(pre_dart.values()), n=5)
        corps = [pre_name[dart] for dart in list_dart_n]
    """
    tokenized_query : keywords
    inside_corp : 언급된 회사
    " ".join(target) : 뉴스텍스트
    corps : topk의 회사이름
    list_dart_n : topk dart
    """
    return tokenized_query, inside_corp, " ".join(target),corps#, list_dart_n


### 3.2. krwordrank with BM25

In [22]:
from krwordrank.word import summarize_with_keywords

def print_corp2(newsindex, dart_dict, bm25, pre_dart, pre_name):
    target_idx=newsindex
    target = df2.iloc[target_idx]
    inside_corp = []

    stopwords=['모두','지난','있다','회의','강조','성과','오는','모집','과정','계약',"체결","건립",'현대식','사업',
            '주관사로','통해','일까지','이벤트','지원','지역','기자','실험','한다','진행','설명회','예정이다',
            '이번','방문','올해','제공','찾아가','재림','당시','한국','프로젝트','있는','추진','다양한','적극',
            '위해','나갈','협력','협약','상당의','물품','명으로','지난해','증가했다','의한','영향','보면','통계',
            '순위','명당','관련','업체','부실','관심','명칭','합작법인','일자리','시흥배곧','서울대','했다',
            '한다고','위원장은','기반','혁신','인재','마련','정책','권고안','함께','시장','성장','대비','전월',
            '전년','동월','물가','하락','소비자','상승','운영','증가','억원','억만원','할인','사용','것으']
    keywords = summarize_with_keywords(target, min_count=4, max_length=7,
        beta=0.85, max_iter=10,stopwords=stopwords,verbose=True)
    tokenized_query = list(keywords.keys())

    for key in ' '.join(target).split(" "):
        if key in dart_dict['corp_name']:
            inside_corp.append(key)

    for key in tokenized_query:
        if key in dart_dict['corp_name']:
            inside_corp.append(key)
    
    inside_corp = list(set(inside_corp))
    
    doc_scores = bm25.get_scores(tokenized_query)
    if max(doc_scores) <5:
        #마땅한 회사가 없음
        return False
    else:
        list_dart_n = bm25.get_top_n(tokenized_query, list(pre_dart.values()), n=5)
        corps = [pre_name[dart] for dart in list_dart_n]
    """
    tokenized_query : keywords
    inside_corp : 언급된 회사
    " ".join(target) : 뉴스텍스트
    corps : topk의 회사이름
    list_dart_n : topk dart
    """
    return tokenized_query, inside_corp, " ".join(target),corps, [e[300:1000] for e  in list_dart_n]

### 3.3 Eval with NER & kwordrank with BM25

In [23]:

#tokenized_corpus = [doc.split(" ") for doc in dart_dict.keys()]
#bm25 = BM25Okapi(tokenized_corpus)
news_id = 7

print_corp(news_id, dart_dict, bm25, pre_dart, pre_name)

[11.7465628  12.63472128  5.82667598 ... 11.19772453 12.19480272
 11.62589758]
33.79406007718258


(['광양', '상공', '회의소', '회장', '광양', '시청', '시장', '시장', '회장', '상', '회장', '기업'],
 [],
 '기업하기 좋은 도시 지역경제 활성화다짐이성훈 최근 취임한 이백구 제4대 광양상공회의소 회장을 비롯한 임원단이 지난 2일 광양시청을 방문 정현복 시장과 만나 지역경제 활성화를 함께 이끌어 나가기로 다짐했다정현복 시장은 이백구 회장의 취임을 축하하며광양에서 기업하기 좋은 경제 환경을 조성하고 소상공인 골목상권 활성화와 시민이 참여하는 경제살리기 운동 등 지역 현안에 대해 실천 결의를 다지는 계기를 함께 마련해 나가자고 말했다이어올해는 시민의 삶이 나아지는 경제운영에 목표를 두고 있다면서 실물경제가 살아나도록 기업은 투자를 확대하고 시민은 지갑을 여는 모두가 행복한 경제 정책을 펼쳐나겠다고 덧붙였다이백구 상의회장은기업이 지역환원사업에 과감히 투자해 시민들로부터 환영받을 수 있도록 최선을 다하겠다며 소상공인들의 안정적인 경영을 통해 골목상권 활성화에 적극 동참하겠다고 밝혔다이번에 취임한 이백구 회장은 김효수 전 회장에 이어 지난 1월 18일부터 광양상공회의소를 이끌어 오고 있다이 회장은 특',
 ['큐라클', '문배철강', '제넥신', '유성티엔에스', '한신공영'])

In [24]:
print_corp2(news_id, dart_dict, bm25, pre_dart, pre_name)

scan vocabs ... 
num vocabs = 38
done = 10


(['회장', '소상공인', '활성화', '시민', '취임', '경제', '기업'],
 [],
 '기업하기 좋은 도시 지역경제 활성화다짐이성훈 최근 취임한 이백구 제4대 광양상공회의소 회장을 비롯한 임원단이 지난 2일 광양시청을 방문 정현복 시장과 만나 지역경제 활성화를 함께 이끌어 나가기로 다짐했다정현복 시장은 이백구 회장의 취임을 축하하며광양에서 기업하기 좋은 경제 환경을 조성하고 소상공인 골목상권 활성화와 시민이 참여하는 경제살리기 운동 등 지역 현안에 대해 실천 결의를 다지는 계기를 함께 마련해 나가자고 말했다이어올해는 시민의 삶이 나아지는 경제운영에 목표를 두고 있다면서 실물경제가 살아나도록 기업은 투자를 확대하고 시민은 지갑을 여는 모두가 행복한 경제 정책을 펼쳐나겠다고 덧붙였다이백구 상의회장은기업이 지역환원사업에 과감히 투자해 시민들로부터 환영받을 수 있도록 최선을 다하겠다며 소상공인들의 안정적인 경영을 통해 골목상권 활성화에 적극 동참하겠다고 밝혔다이번에 취임한 이백구 회장은 김효수 전 회장에 이어 지난 1월 18일부터 광양상공회의소를 이끌어 오고 있다이 회장은 특히 취임식을 생략한 비용으로 지역 소상공인을 대상으로 취임 인사회와 소상공인의 의견을 청취하는 등 지역경제 활성화에 적극 노력하겠다는 의지를 보여 지역민들로부터 큰 호응을 받고 있다2008년 창립한 광양상공회의소는 포스코광양제철소와 협력사 항만 관련 기업 등 300여 업체가 참여한 경제 단체로써 기업성장과 상공인의 건전한 육성을 통한 지역 발전을 이끌어 오고 있다',
 ['한신공영', '덕양산업', '동아엘텍', '에이루트', '쇼박스'],
 ['병에 의한 소멸  에이치케이세종  이관에 의한 소멸  나 회사의 법적상업적 명칭    당사의 명칭은 한신공영 주식회사라고 표기합니다 영문으로는       으로 표기합니다  다 설립일자     당사는 년 월 일에 한신축로공업사라는 상호로 설립되었고 년 월 일 한신공영으로 상호를 변경하여 법인으로 등기하였습니다    년 월 일 한국거래소에 주식을 